# Forcing data generator
**Author: Jun Sasaki Coded on 2025-06-19  Updated on 2025-06-19**<br>
- The input data datetime is supposed to be JST, while using UTC in the script.

In [ ]:
from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    print("This cell is skipped.")


In [ ]:
#%%skip
from pathlib import Path
import pandas as pd
from xfvcom.io.sources.timeseries import TimeSeriesSource

ts = TimeSeriesSource(
    Path("../data/sample_sjis.tsv"),
    interp_method="linear",
)

timeline = pd.date_range("2024-12-31 15:00", periods=5, freq="6h", tz="UTC")
print(timeline)
print(ts.get_series("flux", timeline))

In [ ]:
#%%skip
from pathlib import Path
import pandas as pd
from xfvcom.io.sources.timeseries import TimeSeriesSource

# 任意の sample_sjis.tsv を使用
src = TimeSeriesSource(Path("../data/sample_sjis.tsv"))
timeline = pd.date_range("2025-01-01", periods=3, freq="6h", tz="UTC")
out = src.get("flux", timeline)
print(out)          # 欠落分は NaN
timeline = pd.to_datetime(
    ["2024-12-31 15:00", "2025-01-01 03:00", "2025-01-01 15:00"],
    utc=True,
)
out = src.get_series("flux", timeline)
print(out)   # → array([120.,  nan, 130.])

In [ ]:
#%%skip
from pathlib import Path
from xfvcom.io.sources.utils import load_timeseries_table

# TSV 読み込み（Shift-JIS）
df_tsv = load_timeseries_table(Path("../data/sample_sjis.tsv"))
print(df_tsv.head(), df_tsv.attrs["encoding"])  # => 'cp932'

# 欠損値が NaN になっているか確認
assert df_tsv.isna().any().any()


In [ ]:
#%%skip
from pathlib import Path
from xfvcom.io.met_nc_generator import MetNetCDFGenerator


# --- グリッドのみでプロットターを作る ---------------
dat = Path("~/Github/TB-FVCOM/goto2023/input/TokyoBay18_grd.dat").expanduser()

gen = MetNetCDFGenerator(
    grid_nc   = Path(dat),
    start     = "2025-01-01T00:00:00Z",
    end       = "2025-01-02T00:00:00Z",
    dt_seconds= 3600,
    utm_zone  = 54,        # ★ 必須：UTM ゾーン
    # northern = True,     # 南半球の場合のみ False に変更
    uwind     = 2.0,       # 以降は定数値を自由に上書き
    vwind     = 1.0,
)
nc_path = gen.write(Path('wnd.nc'))      # デフォルト: TokyoBay18_wnd.nc
print("written to", nc_path)

In [ ]:
from pathlib import Path
import xarray as xr
from xfvcom.io.met_nc_generator import MetNetCDFGenerator

# Example grid.dat path
dat = Path("~/Github/TB-FVCOM/goto2023/input/TokyoBay18_grd.dat").expanduser()

# Prepare short_wave time-series table (Asia/Tokyo timestamps)
ts_path = Path("short_wave.csv")
ts_path.write_text(
    "time,short_wave\n"
    "2025-01-01T00:00:00+09:00,300\n"
    "2025-01-01T12:00:00+09:00,250\n"
    "2025-01-02T00:00:00+09:00,280\n",
    encoding="utf-8"
)

gen = MetNetCDFGenerator(
    grid_nc=dat,
    start="2025-01-01T00:00:00Z",
    end="2025-01-02T00:00:00Z",
    start_tz="UTC", # "ASIA/TOKYO", "UTC"
    dt_seconds=3600,
    utm_zone=54,
    ts_specs=[f"{ts_path}:short_wave"],
    data_tz="UTC",
    uwind=2.0,
    vwind=1.0,
)

gen.write(Path("met.nc"))
ds_met = xr.open_dataset("met.nc", decode_times=False)
ds_met

In [ ]:
ds_met.vwind_speed.values

## River data generator
**Note:** When you edit the following cell, restart the notebook.
- Specify time zone `start_tz` and `data_tz`. All datetime is converted to UTC internally and the time zone in the netcdf is also UTC.
- If you want to ignore time zone, specify `start_tz='UTC'` and `data_tz='UTC'`, which will not convert time zone.

In [ ]:
from pathlib import Path
import pandas as pd
import xarray as xr
from xfvcom.io.river_nc_generator import RiverNetCDFGenerator

# 1) Create sample flux data in a DataFrame and save it as a CSV file.
df = pd.DataFrame({
    "time": pd.date_range("2024-12-31 00:00", periods=3, freq="24h"),
    "flux": [100, 105, 110],   # River discharge (m³/s)
})
df.to_csv("arakawa_flux.csv", index=False)

# 2) Create NetCDF using RiverNetCDFGenerator
gen = RiverNetCDFGenerator(
    nml_path    = Path("../configs/rivers_minimal.nml"),
    start       = "2025-01-01T00:00",
    end         = "2025-01-01T12:00",
    start_tz    = "UTC", # "ASIA/TOKYO", # "ASIA/TOKYO" "UTC"
    dt_seconds  = 3600,
    data_tz     = "UTC", # "ASIA/TOKYO", # "ASIA/TOKYO" "UTC"
    ts_specs    = ["Arakawa=arakawa_flux.csv:flux"],  # Use flux column in arakawa_flux.csv file
    const_specs = ["Arakawa.salt=0.05"],              # Use the constant salinity
)

nc_bytes = gen.render()
Path("river.nc").write_bytes(nc_bytes)
ds_river = xr.open_dataset("river.nc", decode_times=False)
ds_river
#ds_river.Times.values